# Lynx Motion AL5D

![](dh_pics/linx_motion_al5d.jpg)

| Dir    | PWM  |
|--------|------|
|CCW_max | 1200 |
|Center  | 1500 |
|CW_max  | 1800 |

In [1]:
%matplotlib inline

In [5]:
from __future__ import print_function
from __future__ import division
import numpy as np
from sympy import symbols, sin, cos, simplify, trigsimp, pi
from math import radians as d2r
from math import degrees as r2d
from math import atan2, sqrt, acos, fabs

In [6]:
class mDH(object):
    """
    This uses the modified DH parameters
    see Craig, eqn (3.6)
    """
    def __init__(self):
        pass

    def fk(self, params):
        t = np.eye(4)
        for p in params:
            t = t.dot(self.makeT(*p))
        return t

    def makeT(self, a, alpha, d, theta):
#         alpha = d2r(alpha)
#         theta = d2r(theta)
        return np.array([  # classic DH
            [           cos(theta),           -sin(theta),           0,             a],
            [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -d*sin(alpha)],
            [sin(theta)*sin(alpha), cos(theta)*sin(alpha),  cos(alpha),  d*cos(alpha)],
            [                    0,                     0,           0,             1]
        ])

def eval(f):
    """
    This allows you to simplify the trigonomic mess that kinematics can
    create and also substitute in some inputs in the process
    """
    c = []
    for row in f:
        r = []
        for col in row:
            # use python symbolic toolbox to simplify the trig mess above 
            r.append(simplify(col))
        c.append(r)
    return np.array(c)

def subs(f, m):
    """
    This allows you to simplify the trigonomic mess that kinematics can
    create and also substitute in some inputs in the process
    """
    c = []
    for row in f:
        r = []
        for col in row:
            r.append(col.subs(m))
        c.append(r)
    return np.array(c)

The DH parameters are:

| i |$a_i$        | $\alpha_i$   | $d_i$   | $theta_i$  |
|---|-------------|--------------|---------|------------|
| 1 | 0           | 0            | $d_2$   | $\theta_1$ |
| 2 | 0           |$\frac{\pi}{2}$| 0       | $\theta_2$ |
| 3 | $a_3$       | 0            | 0       | $\theta_3$ |
| 4 | $a_4$       | 0            | 0       | $\theta_4$ |
| 5 | $a_5$       | 0            | 0       |          0 |

In [15]:
t1, t2, t3, t4 = symbols('t1 t2 t3 t4')
# a2, a3, a4, a5 = symbols('a2 a3 a4 a5')

# t1 - base
# t2 - shoulder
# t3 - elbow
# t4 - wrist

# a2 = 2.75   # base to shoulder
a2 = 0.0
a3 = 5.75   # shoulder to elbow
a4 = 7.375  # elbow to wrist
a5 = 3.375  # wrist to end effector

# put all angles in rads
# sybolic can't handle converting a symbol using d2r()
# a, alpha, d, theta
params = [
    [    0,    0,    a2, t1],
    [    0, pi/2,   0, t2],
    [   a3,    0,    0, t3],
    [   a4,    0,    0, t4],
    [   a5,    0,    0,  0]
]
# params = [
#     [       0,    0, 2.75, t1],
#     [       0, pi/2,    0, t2],
#     [    5.75,    0,    0, t3],
#     [   7.375,    0,    0, t4],
#     [   3.375,    0,    0,  0]
# ]
dh = mDH()
al5d = dh.fk(params)
al5d = eval(al5d)

In [16]:
def printT(tt):
    R = tt[0:3,0:3]
    D = tt[0:3, 3]
    print('-'*30)
    print('Position:')
    print('  x:', D[0])
    print('  y:', D[1])
    print('  z:', D[2])
    # R(n, o, a)
    print('-'*30)
    print('Orientation')
    print('  nx:', R[0,0])
    print('  ny:', R[0,1])
    print('  nz:', R[0,2])
    print('')
    print('  ox:', R[1,0])
    print('  oy:', R[1,1])
    print('  oz:', R[1,2])
    print('')
    print('  ax:', R[2,0])
    print('  ay:', R[2,1])
    print('  az:', R[2,2])

In [17]:
printT(al5d)

------------------------------
Position:
  x: (5.75*cos(t2) + 7.375*cos(t2 + t3) + 3.375*cos(t2 + t3 + t4))*cos(t1)
  y: (5.75*cos(t2) + 7.375*cos(t2 + t3) + 3.375*cos(t2 + t3 + t4))*sin(t1)
  z: 5.75*sin(t2) + 7.375*sin(t2 + t3) + 3.375*sin(t2 + t3 + t4)
------------------------------
Orientation
  nx: 1.0*cos(t1)*cos(t2 + t3 + t4)
  ny: -1.0*sin(t2 + t3 + t4)*cos(t1)
  nz: 1.0*sin(t1)

  ox: 1.0*sin(t1)*cos(t2 + t3 + t4)
  oy: -1.0*sin(t1)*sin(t2 + t3 + t4)
  oz: -1.0*cos(t1)

  ax: 1.0*sin(t2 + t3 + t4)
  ay: 1.0*cos(t2 + t3 + t4)
  az: 0


In [18]:
# dir(al5d)
simp=subs(al5d, [(t1,0.0),(t2,pi/2), (t3, -pi/2), (t4, 0.0)])
printT(simp)

------------------------------
Position:
  x: 10.7500000000000
  y: 0
  z: 5.75000000000000
------------------------------
Orientation
  nx: 1.00000000000000
  ny: 0
  nz: 0

  ox: 0
  oy: 0
  oz: -1.00000000000000

  ax: 0
  ay: 1.00000000000000
  az: 0


In [23]:
simp=subs(al5d, [(t1,0.0),(t2,d2r(111.5)), (t3, d2r(-127)), (t4, d2r(-74.5))])
printT(simp)

------------------------------
Position:
  x: 4.99939253874888
  y: 0
  z: 0.00401799231949518
------------------------------
Orientation
  nx: -1.60812264967664e-16
  ny: 1.00000000000000
  nz: 0

  ox: 0
  oy: 0
  oz: -1.00000000000000

  ax: -1.00000000000000
  ay: -1.60812264967664e-16
  az: 0


# Inverse Kinematics

In [11]:
ca, sa, px, py, pz = symbols('ca sa px py pz')
desired = np.array([
    [ ca,  0, sa, px],
    [-sa,  0, ca, py],
    [  0, -1,  0, pz],
    [  0,  0,  0,  1]
])

In [20]:
params = [
    [   0.0, pi/2,  0, t2],
    [  5.75,    0,  0, t3],
    [ 7.375,    0,  0, t4],
    [ 3.375,    0,  0,  0]
]
dh = mDH()
plane = dh.fk(params)
plane = eval(plane)

In [21]:
printT(plane)

------------------------------
Position:
  x: 5.75*cos(t2) + 7.375*cos(t2 + t3) + 3.375*cos(t2 + t3 + t4)
  y: 0
  z: 5.75*sin(t2) + 7.375*sin(t2 + t3) + 3.375*sin(t2 + t3 + t4)
------------------------------
Orientation
  nx: 1.0*cos(t2 + t3 + t4)
  ny: -1.0*sin(t2 + t3 + t4)
  nz: 0

  ox: 0
  oy: 0
  oz: -1.00000000000000

  ax: 1.0*sin(t2 + t3 + t4)
  ay: 1.0*cos(t2 + t3 + t4)
  az: 0


In [13]:
a2+a3+a4+a5

19.25

# Alt

![](http://mathworld.wolfram.com/images/eps-gif/LawofCosines_1000.gif)

$$
a^2 = b^2 + c^2 - 2bc \cos(A) \rightarrow \cos(A)=\frac{-a^2+b^2+c^2}{2bc}\\
b^2 = a^2 + c^2 - 2ac \cos(B) \rightarrow \cos(B)=\frac{a^2-b^2+c^2}{2ac}\\
c^2 = a^2 + b^2 - 2ab \cos(C) \rightarrow \cos(C)=\frac{a^2+b^2-c^2}{2ab}
$$

[law cosines](http://mathworld.wolfram.com/LawofCosines.html)

In [53]:
from math import acos, asin, atan2, pi, sqrt

x,y,z = (1,-5,0)

theta1 = atan2(y,x)

In [54]:
gripper = 3.375
zz = z + gripper
b = 5.75
a = 7.375
g = sqrt(x**2+y**2)
c = sqrt(g**2+zz**2)
gamma = atan2(zz, g)
A = acos((-a**2+b**2+c**2)/(2*b*c))
theta2 = A+gamma
theta3 = acos((a**2+b**2-c**2)/(2*a*b))

print('theta_1: {}'.format(theta1*180/pi))
print('theta_2: {}'.format(theta2*180/pi))
print('theta_3: {}'.format(theta3*180/pi))

theta_1: -78.69006752597979
theta_2: 110.29619226285975
theta_3: 53.823640611457044


In [55]:
p=b*cos(theta2)
q=b*sin(theta2)
m=q-z
n=c-p
r=sqrt(a**2+gripper**2)
theta4 = acos((a**2+gripper**2-r**2)/(2*a*gripper))
print('theta_4: {}'.format(theta4*180/pi))

theta_4: 90.0


In [45]:
121-(180-70)

11